# Journée Axe Bioinformatique

**Thème** : outils et environnements FAIR pour la bioanalyse

Thomas Denecker - IFB core

## Contexte

## Demo

### Création d'un répertoire de travail

In [ ]:
mkdir demo
cd demo 

### Import des données

**Téléchargement des données**

In [ ]:
wget https://zenodo.org/record/3997237/files/FAIR_Bioinfo_data.tar.gz?download=1 -O FAIR_Bioinfo_data.tar.gz

**Extraction des données**

In [ ]:
tar -xvzf FAIR_Bioinfo_data.tar.gz

### Récupération des scripts d'analyse
**Snakemake files**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/config.yml
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/demo.smk

**Slurm files**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/runSARtools.slurm
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/runSnakemake.slurm

**R files**

In [ ]:
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/condition.txt
wget https://raw.githubusercontent.com/IFB-ElixirFr/journeeAxeBioinfo_2020/main/sartools.R

### Lancement de l'analyse

In [ ]:
sbatch runSnakemake.slurm

In [ ]:
SINGULARITY_LOCALCACHEDIR=${PWD}
SINGULARITY_PULLFOLDER=${PWD}
singularity pull docker://tdenecker/journee_axe_bioinfo

In [ ]:
sbatch runSARtools.slurm